Fine-tuning with labels copied from input ids

In [1]:
!pip install -U transformers datasets peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl 

In [4]:
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 16.1 MB/s eta 0:00:00


In [5]:
from copy import copy
from time import time
from typing import Dict, List, Optional

import pandas as pd
import torch
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from peft.peft_model import PeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BatchEncoding,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from trl import DataCollatorForCompletionOnlyLM, SFTTrainer

In [6]:
INSTRUCTION_TEMPLATE_BASE = "\n\n### Human:"
RESPONSE_TEMPLATE_BASE = "\n\n### Assistant:"


def load_model(
    model_name: str, peft_kwargs: Optional[Dict] = None
) -> PeftModelForCausalLM:
    model = AutoModelForCausalLM.from_pretrained(model_name)
    if peft_kwargs is None:
        peft_kwargs = {}
    peft_config = LoraConfig(task_type="CAUSAL_LM", **peft_kwargs)
    # alterantively, you can use the following to load the model
    # model = PeftModelForCausalLM.from_pretrained(model_name)
    model = get_peft_model(model, peft_config)
    return model


def add_special_tokens(
    example: Dict,
    tokenizer: PreTrainedTokenizerBase,
) -> Dict:
    # add eos_token before human text and bos_token before assistant text
    example["text"] = (
        example["text"]
        .replace(
            INSTRUCTION_TEMPLATE_BASE, tokenizer.eos_token + INSTRUCTION_TEMPLATE_BASE
        )
        .replace(RESPONSE_TEMPLATE_BASE, RESPONSE_TEMPLATE_BASE + tokenizer.bos_token)
    )
    if not example["text"].endswith(tokenizer.eos_token):
        example["text"] += tokenizer.eos_token
    # Remove leading EOS tokens
    while example["text"].startswith(tokenizer.eos_token):
        example["text"] = example["text"][len(tokenizer.eos_token) :]

    return example

In [7]:
# Preprocessing
model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True, padding_side="right"
)  # padding side should be right for causal lm

# overfit to 5 examples
str1 = '\n\n### Human: How do you say "dog" in Spanish?\n\n### Assistant: perro'
str2 = '\n\n### Human: How do you say "water" in Spanish?\n\n### Assistant: agua'
str3 = '\n\n### Human: How do you say "mother" in Spanish?\n\n### Assistant: madre'
str4 = '\n\n### Human: How do you say "hello" in Spanish?\n\n### Assistant: hola'
str5 = '\n\n### Human: How do you say "tree" in Spanish?\n\n### Assistant: árbol'
train_data = {
    "text": [str1, str2, str3, str4, str5],
}
dataset_text = Dataset.from_dict(train_data)
dataset_text = dataset_text.map(lambda x: add_special_tokens(x, tokenizer))
print(f"{dataset_text=}")
print(f"{dataset_text[0]=}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

dataset_text=Dataset({
    features: ['text'],
    num_rows: 5
})
dataset_text[0]={'text': '\n\n### Human: How do you say "dog" in Spanish?\n\n### Assistant:<s> perro</s>'}


In [8]:
# tokenize the text
dataset = dataset_text.map(
    lambda example: tokenizer(example["text"]), batched=True, remove_columns=["text"]
)
# copy the input_ids to labels
dataset = dataset.map(lambda x: {"labels": x["input_ids"]}, batched=True)
print(f"{dataset=}")
print(f"{dataset[0]['input_ids']=}")
print(f"{dataset[0]['labels']=}")
print(f"{dataset[0]['attention_mask']=}")

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

dataset=Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5
})
dataset[0]['input_ids']=[603, 105311, 22256, 29, 7535, 727, 1152, 5894, 20587, 744, 5, 361, 49063, 7076, 105311, 143005, 29, 1, 82208, 2]
dataset[0]['labels']=[603, 105311, 22256, 29, 7535, 727, 1152, 5894, 20587, 744, 5, 361, 49063, 7076, 105311, 143005, 29, 1, 82208, 2]
dataset[0]['attention_mask']=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
# training code inspired by
# https://mlabonne.github.io/blog/posts/Fine_Tune_Your_Own_Llama_2_Model_in_a_Colab_Notebook.html

model = load_model(model_name)
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# How many times to iterate over the entire dataset
num_train_epochs = 15

# We're not aligning the sequence length (ie padding or truncating)
# so batch training won't work for our toy example.
per_device_train_batch_size = 1


training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    seed=1,
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [10]:
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=training_arguments,
)

In [11]:
holdout_str = (
    '\n\n### Human: How do you say "good" in Spanish?\n\n### Assistant:<s>'  # bueno
)
device = "cuda" if torch.cuda.is_available() else "cpu"
holdout_input = tokenizer(holdout_str, return_tensors="pt").to(device)

In [12]:
def sample_generate(
    model: torch.nn.Module,
    tokenizer: PreTrainedTokenizerBase,
    inputs: BatchEncoding,
    **kwargs
) -> str:
    """Runs tokenized text through the model and returns the generated text."""
    outputs = model.generate(**inputs, **kwargs)
    gen_text = tokenizer.batch_decode(
        # strip the text of the prompt
        outputs[:, inputs["input_ids"].shape[1] :]
    )
    return gen_text[0]

In [13]:
ENGLISH_WORDS = ["dog", "water", "mother", "hello", "tree"]
SPANISH_WORDS = ["perro", "agua", "madre", "hola", "árbol"]


def predict_training_set(
    model: torch.nn.Module,
    tokenizer: PreTrainedTokenizerBase,
    english_words: List[str] = ENGLISH_WORDS,
    spanish_words: List[str] = SPANISH_WORDS,
):
    """Runs predictions on the entire training set."""
    for eng, span in zip(english_words, spanish_words):
        inputs2 = tokenizer(
            f'\n\n### Human: How do you say "{eng}" in Spanish?\n\n### Assistant:<s>',
            return_tensors="pt",
        ).to(device)
        print(
            "real answer:",
            span,
            "\tpredicted answer:",
            sample_generate(model, tokenizer, inputs2, max_new_tokens=5),
        )

In [14]:
def print_iterative_generate(model, tokenizer, inputs):
    """Approximates the training forward pass by iterating through a sequence
    and predicting one token at a time.
    """
    tok_outputs = []
    for tok_id in range(1, len(inputs["input_ids"][0]) + 1):
        iterative_inputs = inputs.copy()
        iterative_inputs["input_ids"] = inputs["input_ids"][:, :tok_id]
        iterative_inputs["attention_mask"] = inputs["attention_mask"][:, :tok_id]
        tok_outputs.append(
            sample_generate(model, tokenizer, iterative_inputs, max_new_tokens=1)
        )
    print("".join(tok_outputs))

In [15]:
predict_training_set(model, tokenizer)

real answer: perro 	predicted answer: </s>
real answer: agua 	predicted answer: </s>
real answer: madre 	predicted answer: </s>
real answer: hola 	predicted answer: </s>
real answer: árbol 	predicted answer: </s>


In [16]:
original_output = sample_generate(model, tokenizer, holdout_input, max_new_tokens=5)
original_output

'</s>'

In [17]:
print_iterative_generate(model, tokenizer, holdout_input)

#include
 code
 to I get "it's" to a?
A: Spanish: How</s>


In [36]:
training1 = trainer.train()

Step,Training Loss


In [31]:
predict_training_set(model, tokenizer)

real answer: perro 	predicted answer:  perro</s>
real answer: agua 	predicted answer:  agua</s>
real answer: madre 	predicted answer:  madre</s>
real answer: hola 	predicted answer:  hola</s>
real answer: árbol 	predicted answer:  árbol</s>


In [32]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=75, training_loss=0.09987699508666992, metrics={'train_runtime': 7.086, 'train_samples_per_second': 10.584, 'train_steps_per_second': 10.584, 'total_flos': 2700620144640.0, 'train_loss': 0.09987699508666992, 'epoch': 15.0})

In [33]:
predict_training_set(model, tokenizer)

real answer: perro 	predicted answer:  perro</s>
real answer: agua 	predicted answer:  agua</s>
real answer: madre 	predicted answer:  madre</s>
real answer: hola 	predicted answer:  hola</s>
real answer: árbol 	predicted answer:  árbol</s>


In [34]:
sample_generate(model, tokenizer, holdout_input, max_new_tokens=5)

' bueno</s>'

In [35]:
print_iterative_generate(model, tokenizer, holdout_input)

### Human: How do you say "water" in Spanish?

### Assistant:<s> bueno
